<a href="https://colab.research.google.com/github/alvinastr/Everin/blob/main/Kalkulator_BMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [ ]:
file_path = '/content/drive/MyDrive/data/Dataset_Makanan_dan_Minuman.csv'
df = pd.read_csv(file_path, sep=',', encoding='latin-1')

df.columns = df.columns.str.strip()

print("Nama kolom setelah di-strip:")
print(df.columns)

print("Beberapa baris pertama dari DataFrame:")
print(df.head())

Nama kolom setelah di-strip:
Index(['Nama Makanan/Minuman', 'Kalori (kcal)', 'Karbohidrat (g)',
       'Protein (g)', 'Lemak (g)'],
      dtype='object')
Beberapa baris pertama dari DataFrame:
  Nama Makanan/Minuman  Kalori (kcal)  Karbohidrat (g)  Protein (g)  Lemak (g)
0        ABC Kopi Susu             60             10.0          1.0        2.0
1              Acaraki             50             12.0          0.0        0.0
2                 Apel             52             14.0          0.3        0.2
3                Bakso            150             10.0         10.0        8.0
4        Beef Teriyaki            200             15.0         20.0        8.0


In [ ]:
if 'Nama Makanan/Minuman' in df.columns and 'Kalori (kcal)' in df.columns:
    print("Kolom 'Nama Makanan/Minuman' dan 'Kalori (kcal)' ditemukan!")
else:
    print("Kolom 'Nama Makanan/Minuman' atau 'Kalori (kcal)' tidak ditemukan.")

Kolom 'Nama Makanan/Minuman' dan 'Kalori (kcal)' ditemukan!


In [ ]:
try:
    sample_data = df[['Nama Makanan/Minuman', 'Kalori (kcal)']].head()
    print("Beberapa contoh data:")
    print(sample_data)
except KeyError as e:
    print(f"KeyError: Kolom tidak ditemukan - {e}")

Beberapa contoh data:
  Nama Makanan/Minuman  Kalori (kcal)
0        ABC Kopi Susu             60
1              Acaraki             50
2                 Apel             52
3                Bakso            150
4        Beef Teriyaki            200


# Kalkulator & Model

In [ ]:
# Fungsi untuk menghitung BMR dan TDEE
def hitung_bmr_tdee(berat_badan, tinggi_badan, umur, jenis_kelamin, tingkat_aktivitas):
    if jenis_kelamin == 'pria':
        bmr = 10 * berat_badan + 6.25 * tinggi_badan - 5 * umur + 5
    else:  # wanita
        bmr = 10 * berat_badan + 6.25 * tinggi_badan - 5 * umur - 161

    if tingkat_aktivitas == "ringan":
        tdee = bmr * 1.375
    elif tingkat_aktivitas == "sedang":
        tdee = bmr * 1.55
    elif tingkat_aktivitas == "berat":
        tdee = bmr * 1.725
    else:
        raise ValueError("Tingkat aktivitas tidak valid. Pilih 'ringan', 'sedang', atau 'berat'.")

    return bmr, tdee

# Fungsi untuk menghitung kebutuhan makronutrien berdasarkan TDEE
def hitung_kebutuhan_makronutrien(tdee):
    kalori_karbohidrat = tdee * 0.55
    kalori_protein = tdee * 0.20
    kalori_lemak = tdee * 0.25

    gram_karbohidrat = kalori_karbohidrat / 4
    gram_protein = kalori_protein / 4
    gram_lemak = kalori_lemak / 9

    return gram_karbohidrat, gram_protein, gram_lemak

# Fungsi untuk menghitung kalori per bahan makanan
def kalori(Bahan, Gram, df):
    try:
        # Menemukan kalori bahan makanan
        kalori_bahan = df.loc[df['Nama Makanan/Minuman'].str.lower() == Bahan.strip().lower(), 'Kalori (kcal)']
        if kalori_bahan.empty:
            raise ValueError(f"Bahan '{Bahan}' tidak ditemukan dalam database.")
        kalori_bahan = kalori_bahan.iloc[0]
        Kalori = kalori_bahan * Gram
        return Kalori
    except ValueError as e:
        print(f"Error: {e}")
        return 0

# Fungsi untuk melatih model k-NN dan memberikan rekomendasi makanan
def rekomendasi_makanan_ml(df, kebutuhan_karbohidrat, kebutuhan_protein, kebutuhan_lemak):
    # Menyiapkan data untuk training
    X = df[['Karbohidrat (g)', 'Protein (g)', 'Lemak (g)']].values  # Fitur: Karbohidrat, Protein, Lemak
    y = df['Nama Makanan/Minuman'].values  # Target: Nama Makanan

    # Menggunakan k-NN untuk menemukan makanan yang paling sesuai
    model = NearestNeighbors(n_neighbors=5, metric='euclidean')
    model.fit(X)

    # Data input untuk prediksi
    input_data = np.array([[kebutuhan_karbohidrat, kebutuhan_protein, kebutuhan_lemak]])

    # Mencari 5 makanan terdekat berdasarkan kebutuhan makronutrien
    distances, indices = model.kneighbors(input_data)

    print("\nRekomendasi makanan yang sesuai dengan kebutuhan harian Anda:")
    for i in indices[0]:
        makanan = df.iloc[i]
        print(f"{makanan['Nama Makanan/Minuman']}: {makanan['Karbohidrat (g)']}g Karbohidrat, "
              f"{makanan['Protein (g)']}g Protein, {makanan['Lemak (g)']}g Lemak")

# Fungsi utama untuk kalkulator kebutuhan harian
def kalkulator_kebutuhan_harian(df):
    print("Selamat datang di kalkulator kebutuhan kalori harian!")

    berat_badan = float(input("Masukkan berat badan (kg): "))
    tinggi_badan = float(input("Masukkan tinggi badan (cm): "))
    umur = int(input("Masukkan umur (tahun): "))
    jenis_kelamin = input("Masukkan jenis kelamin (pria/wanita): ").lower()
    tingkat_aktivitas = input("Masukkan tingkat aktivitas (ringan/sedang/berat): ").lower()

    try:
        bmr, tdee = hitung_bmr_tdee(berat_badan, tinggi_badan, umur, jenis_kelamin, tingkat_aktivitas)
    except ValueError as e:
        print(f"Error: {e}")
        return

    print(f"\nBMR Anda adalah {bmr:.2f} kalori/hari.")
    print(f"TDEE Anda adalah {tdee:.2f} kalori/hari.")

    gram_karbohidrat, gram_protein, gram_lemak = hitung_kebutuhan_makronutrien(tdee)

    print(f"\nKebutuhan harian Anda adalah:")
    print(f"Karbohidrat: {gram_karbohidrat:.2f} gram")
    print(f"Protein: {gram_protein:.2f} gram")
    print(f"Lemak: {gram_lemak:.2f} gram")

    # Input makanan yang sudah dimakan
    total_kalori_dikonsumsi = 0
    total_karbohidrat_dikonsumsi = 0
    total_protein_dikonsumsi = 0
    total_lemak_dikonsumsi = 0

    while True:
        try:
            Bahan = str(input("Masukkan nama bahan makanan yang sudah Anda makan: ")).lower().strip()
            Gram = int(input("Berapa berat (gram) bahan yang Anda makan? "))

            # Menghitung kalori dan makronutrien makanan yang dimakan
            kalori_makanan = kalori(Bahan, Gram, df)
            if kalori_makanan == 0:
                print(f"Bahan '{Bahan}' tidak ditemukan. Silakan coba lagi.")
                continue

            # Mendapatkan makronutrien
            makanan_row = df.loc[df['Nama Makanan/Minuman'].str.lower() == Bahan.strip().lower()]

            karbohidrat = makanan_row['Karbohidrat (g)'].values[0] * Gram / 100
            protein = makanan_row['Protein (g)'].values[0] * Gram / 100
            lemak = makanan_row['Lemak (g)'].values[0] * Gram / 100

            # Menambahkan konsumsi makanan ke total
            total_kalori_dikonsumsi += kalori_makanan
            total_karbohidrat_dikonsumsi += karbohidrat
            total_protein_dikonsumsi += protein
            total_lemak_dikonsumsi += lemak

            print(f"Kalori yang telah dikonsumsi dari {Bahan}: {kalori_makanan:.2f} kal")
            print(f"Total kalori dikonsumsi: {total_kalori_dikonsumsi:.2f} kal")
            print(f"Karbohidrat yang telah dikonsumsi: {total_karbohidrat_dikonsumsi:.2f} g")
            print(f"Protein yang telah dikonsumsi: {total_protein_dikonsumsi:.2f} g")
            print(f"Lemak yang telah dikonsumsi: {total_lemak_dikonsumsi:.2f} g")

            # Menghitung kebutuhan sisa setelah konsumsi
            sisa_karbohidrat = gram_karbohidrat - total_karbohidrat_dikonsumsi
            sisa_protein = gram_protein - total_protein_dikonsumsi
            sisa_lemak = gram_lemak - total_lemak_dikonsumsi

            print(f"\nSisa kebutuhan karbohidrat: {sisa_karbohidrat:.2f} gram")
            print(f"Sisa kebutuhan protein: {sisa_protein:.2f} gram")
            print(f"Sisa kebutuhan lemak: {sisa_lemak:.2f} gram")

            # Rekomendasi makanan berdasarkan kebutuhan yang tersisa
            rekomendasi_makanan_ml(df, sisa_karbohidrat, sisa_protein, sisa_lemak)

            # Tanya apakah ingin melanjutkan memasukkan makanan lagi
            lanjutkan = input("Apakah Anda ingin memasukkan makanan lain? (y/n): ")
            if lanjutkan.lower() != 'y':
                break

        except ValueError:
            print("Input tidak valid. Harap masukkan nilai yang benar.")
            continue

# Membaca file CSV yang berisi bahan makanan dan kalori
file_path = '/content/drive/MyDrive/data/Dataset_Makanan_dan_Minuman.csv'

# Membaca CSV menjadi DataFrame
df = pd.read_csv(file_path, sep=',', encoding='latin-1')

# Membersihkan nama kolom dari spasi atau karakter tersembunyi
df.columns = df.columns.str.strip()

# Menampilkan data yang telah dimuat untuk memastikan semuanya benar
print(df.head())

# Menjalankan kalkulator
kalkulator_kebutuhan_harian(df)


  Nama Makanan/Minuman  Kalori (kcal)  Karbohidrat (g)  Protein (g)  Lemak (g)
0        ABC Kopi Susu             60             10.0          1.0        2.0
1              Acaraki             50             12.0          0.0        0.0
2                 Apel             52             14.0          0.3        0.2
3                Bakso            150             10.0         10.0        8.0
4        Beef Teriyaki            200             15.0         20.0        8.0
Selamat datang di kalkulator kebutuhan kalori harian!


In [ ]:
import pickle

# Menyimpan model ke file
with open('model_kalkulator.pkl', 'wb') as file:
    pickle.dump([hitung_bmr_tdee, hitung_kebutuhan_makronutrien, kalori, rekomendasi_makanan_ml], file)

# Menyimpan DataFrame juga jika perlu
df.to_csv('data_makanan.csv', index=False)